## 1. DLinear 모델 

DLinear 모델은 시계열 데이터의 예측을 위해 설계된 딥러닝 모델

주로 롱 시퀀스 예측(Long Sequence Forecasting) 문제에서 사용됩니다. 

"DLinear"는 "Decomposition-based Linear Model"의 약자

시계열 데이터를 분해하여 보다 단순한 선형 회귀를 통해 예측하는 접근 방식

이 모델은 특히 복잡한 모델이 오히려 예측 성능을 저하할 수 있는 긴 시계열 문제에서 효과적인 성능을 보이는 것으로 알려져 있습니다.

In [ ]:
# DLinear 클래스는 input_size를 받아서 output_size로 변환하는 두 개의 선형 레이어 (trend_linear와 seasonality_linear)를 가지고 있습니다.
# 모델은 입력 데이터를 **추세(trend)**와 **계절성(seasonality)**으로 분해하여 예측하는 방식으로 동작합니다.
# forward 함수는 주어진 입력 x를 추세와 계절성 레이어에 각각 통과시키고, 그 결과를 합산하여 최종 예측 값을 만듭니다.

In [ ]:
# 데이터 준비
# input_size는 10이고, output_size는 1입니다. 즉, 입력 데이터의 길이는 10이며, 이를 하나의 출력 값으로 예측하려고 합니다.
# x는 크기 (1, 10)인 난수로 생성된 입력 데이터입니다. 이는 배치 크기 1, 길이 10인 벡터입니다.

In [ ]:
# 모델 및 학습 설정
# DLinear 객체 dmodel을 생성하여 모델을 초기화합니다.
# nn.MSELoss()는 평균 제곱 오차(MSE) 손실 함수를 사용해 모델의 예측 값과 실제 값 사이의 오차를 계산합니다.
# optim.SGD는 **확률적 경사하강법(SGD)**을 사용해 모델의 파라미터를 업데이트하는 최적화기입니다. 학습률은 0.01로 설정되어 있습니다.

In [ ]:
# 단일 학습 스텝 실행
# y_true는 모델이 예측하고자 하는 실제 값으로, 크기 (1, 1)인 난수로 생성되었습니다.
# optimizer.zero_grad()는 이전 학습에서 계산된 기울기를 초기화합니다.
# y_pred = dmodel(x)는 모델을 통해 예측 값을 계산합니다.
# loss = criterion(y_pred, y_true)를 통해 예측 값과 실제 값 사이의 손실을 계산합니다.
# loss.backward()는 손실 함수에 대한 기울기를 계산하는 과정입니다. 이를 통해 각 파라미터의 변화 방향을 알 수 있습니다.
# optimizer.step()은 앞에서 계산된 기울기를 사용해 모델의 파라미터를 업데이트합니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class DLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(DLinear, self).__init__()
        # 선형 레이어를 사용하여 추세와 계절성을 학습합니다.
        self.trend_linear = nn.Linear(input_size, output_size)
        self.seasonality_linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        # 입력 x를 추세와 계절성으로 분해하여 예측
        trend = self.trend_linear(x)
        seasonality = self.seasonality_linear(x)
        return trend + seasonality

# 데이터 예시 (배치 크기: 1, 입력 길이: 10)
input_size = 10
output_size = 1
x = torch.randn(1, input_size)

dmodel = DLinear(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(dmodel.parameters(), lr=0.01)

# 학습 예시 (단일 스텝)
y_true = torch.randn(1, output_size)  # 실제 값
optimizer.zero_grad()
y_pred = dmodel(x)  # 예측 값
loss = criterion(y_pred, y_true)  # 손실 계산
loss.backward()  # 역전파
optimizer.step()  # 가중치 업데이트

print(f'예측 값: {y_pred.item()}, 손실 값: {loss.item()}')


예측 값: 0.33053064346313477, 손실 값: 0.21795296669006348


In [ ]:
# 출력 결과

# print(f'예측 값: {y_pred.item()}, 손실 값: {loss.item()}')는 학습 후 예측 값(y_pred)과 손실 값(loss)을 출력합니다.
# 예측 값: 모델이 입력 데이터를 바탕으로 계산한 예측 값입니다.
# 손실 값: 예측 값과 실제 값 사이의 오차를 의미하며, 이 값이 작을수록 모델의 예측이 정확하다는 것을 나타냅니다.

In [ ]:
예측 값의 의미

예측 값 (y_pred.item()):  이 값은 삼성전자의 주가 수익률을 예측한 결과입니다. 

    구체적으로, 최근 10일간의 수익률 데이터를 입력으로 받아서 다음 날의 수익률(즉, 종가가 얼마나 변동할지를 비율로 나타낸 값)을 예측한 것입니다.

예를 들어, 예측 값이 0.33이면 이는 주가가 다음 날 33% 상승할 것으로 예상한다는 의미입니다. 반대로 예측 값이 -0.02라면 주가가 2% 하락할 것으로 예측하는 것입니다.

## 2. 삼성전자 일별 데이터

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import yfinance as yf
import numpy as np

class DLinear(nn.Module):
    def __init__(self, input_size, output_size):
        super(DLinear, self).__init__()
        # 선형 레이어를 사용하여 추세와 계절성을 학습합니다.
        self.trend_linear = nn.Linear(input_size, output_size)
        self.seasonality_linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        # 입력 x를 추세와 계절성으로 분해하여 예측
        trend = self.trend_linear(x)
        seasonality = self.seasonality_linear(x)
        return trend + seasonality

# 삼성전자 주가 데이터 가져오기
ticker = '005930.KS'
start_date = '2023-01-01'
end_date = '2024-11-19'

df = yf.download(ticker, start=start_date, end=end_date)

# 종가 데이터를 사용하여 입력 데이터 준비
df['Return'] = df['Close'].pct_change()
df = df.dropna()

# 시계열 데이터 준비 (배치 크기: 1, 입력 길이: 10)
input_size = 10
output_size = 1

# 최근 10일간의 수익률을 입력으로 사용
x = torch.tensor(df['Return'].values[-input_size:], dtype=torch.float32).view(1, -1)

# DLinear 모델 초기화 및 학습 설정
dmodel = DLinear(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(dmodel.parameters(), lr=0.01)

# 실제 값 (다음 날의 수익률을 예측)
y_true = torch.tensor([df['Return'].values[-1]], dtype=torch.float32).view(1, -1)

# 학습 예시 (단일 스텝)
optimizer.zero_grad()
y_pred = dmodel(x)  # 예측 값
loss = criterion(y_pred, y_true)  # 손실 계산
loss.backward()  # 역전파
optimizer.step()  # 가중치 업데이트

print(f'예측 값: {y_pred.item()}, 손실 값: {loss.item()}')


[*********************100%***********************]  1 of 1 completed

예측 값: 0.22544682025909424, 손실 값: 0.027434535324573517
